Passive Learning Test
(M=1)

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings("ignore")

X_y_train = pd.read_csv('Bank_Train.csv',index_col=0)
print(X_y_train)
X_y_test = pd.read_csv('Bank_Test.csv',index_col=0)
print(X_y_test)
X_test = X_y_test.drop(columns=['y'])
y_test = X_y_test['y']
print(X_test)
print(y_test)

          X1        X2       X3       X4  y
0    1.35660   4.23580  2.13410  0.32110  0
1    0.51950  -3.26330  3.08950 -0.98490  0
2   -1.11930  10.72710  2.09380 -5.65040  0
3    1.43780   0.66837 -2.02670  1.02710  1
4   -2.01490   3.68740 -1.93850 -3.89180  1
..       ...       ...      ...      ... ..
895  1.25720   4.87310 -5.28610 -5.87410  1
896 -0.82053   0.65181 -0.48869 -0.52716  1
897 -1.66370   3.28810 -2.27010 -2.22240  1
898  4.36340   0.46351  1.42810  2.02020  0
899 -2.19790  -2.12520  1.71510  0.45171  1

[900 rows x 5 columns]
          X1        X2       X3       X4  y
0   -7.03640  9.293100  0.16594 -4.53960  1
1    0.53936  3.894400 -4.81660 -4.34180  1
2   -0.49281  3.060500 -1.83560 -2.83400  1
3   -0.83121  0.039307  0.05369 -0.23105  1
4    1.67990  4.206800 -4.53980 -2.39310  1
..       ...       ...      ...      ... ..
467 -0.56877  1.417400 -1.42520 -1.12460  1
468 -5.49010  9.104800 -0.38758 -5.97630  1
469  0.85574  0.008268  6.60420 -0.53104  0
470  3.4

In [15]:
X_y_train_updated = X_y_train.copy()                  # to save the original X_y_train

X_y_train_build = pd.DataFrame()                      # defining an empty dataframe where actual train samples will be stored

C_range = np.array([10**(-3),10**(-2),10**(-1),10**(0),10**(1),10**(2),10**(3),10**(4),10**(5),10**(6)])

Error_Test_df = pd.DataFrame(index=np.arange(1,2),columns=np.arange(10,910,10))

# this range is typical, and is assumed. Range selection, as done previously, may be performed for penalty to obtain its upper
# and lower limits, but since this range is usually typical, and since it has already been done once for a previous section, we
# will assumed this C range rather than again seperately performing range selection here
# As is, this is a sufficient enough range, as is the typical range the penalty usually lies in for SVM

for i in np.arange(1,91):          # Directly
    
    if i != 90:
        X_y_train_to_add, X_y_train_updated = train_test_split(X_y_train_updated,train_size=10,shuffle=True,stratify=X_y_train_updated['y'])
        X_y_train_to_add.reset_index(drop=True,inplace=True)
        X_y_train_updated.reset_index(drop=True,inplace=True)
        
    else:
        X_y_train_to_add = X_y_train_updated
    
    
    
    X_y_train_build = pd.concat([X_y_train_build,X_y_train_to_add],axis=0)
    
    X_y_train_build.reset_index(drop=True,inplace=True)
    
    
    
    CV_errors_df = pd.DataFrame(index=np.arange(0,5),columns=C_range) 
    
    cv_ite = -1
    
    X_train_build = X_y_train_build.drop(columns=['y'])
    y_train_build = X_y_train_build['y']
    
    
    kf = KFold(n_splits=5,shuffle=True)        # Normal KFold
                                               # As such if needed it is easy to perform stratified KFold as well
    
    for train,test in kf.split(X_train_build):
        
        cv_ite = cv_ite + 1
        
        X_train_cv, X_test_cv = X_train_build.iloc[train,:],X_train_build.iloc[test,:]
        y_train_cv, y_test_cv = y_train_build.iloc[train],y_train_build.iloc[test]
        
        
        for j in np.arange(0,C_range.size):
            
            svm_cv = LinearSVC(penalty='l1',dual=False,C=C_range[j]).fit(X_train_cv,y_train_cv)
            
            y_test_cv_predicted = svm_cv.predict(X_test_cv)
            
            mis_loc = 0
            
            for l in np.arange(0,X_test_cv.shape[0]):
                
                if y_test_cv_predicted[l] != y_test_cv.iloc[l]:
                    
                    mis_loc = mis_loc + 1
                    
            err_loc = (mis_loc/X_test_cv.shape[0])*100
            
            CV_errors_df.iloc[cv_ite,j] = err_loc
            
    
    
    CV_mean_errors = CV_errors_df.mean(axis=0)
    
    error_vector = CV_mean_errors.to_numpy().flatten()
    
    indx_C_selected = np.argmin(error_vector)
    
    min_C = C_range[indx_C_selected]
        
    
    
    svm_f = LinearSVC(penalty='l1',dual=False,C=min_C).fit(X_train_build,y_train_build)
    
    y_test_predicted = svm_f.predict(X_test)
    
    mis = 0
    
    for k in np.arange(0,X_test.shape[0]):
        
        if y_test_predicted[k] != y_test.iloc[k]:
            
            mis = mis + 1
        
    err = (mis/X_test.shape[0])*100
    
    
    Error_Test_df.iloc[0,i-1] = err
    
    

    
    
print(Error_Test_df)    

       10       20       30       40       50       60      70       80   \
1  11.0169  10.3814  2.75424  2.75424  2.75424  2.75424  2.9661  1.90678   

       90       100  ...      810      820      830      840      850  \
1  2.11864  2.33051  ...  1.05932  1.05932  1.05932  1.05932  1.48305   

       860      870      880      890      900  
1  1.05932  1.05932  1.05932  1.05932  1.05932  

[1 rows x 90 columns]


The above result is for a single M=1 iteration, where all the 90 errors have been obtained